# PyTorch tutorial

簡単な例を以下に示す（モデルは https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html 参照）。


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

`Module`から派生させたクラスには、レイヤーとforward propの順番（＝モデルをどう組み立てるか）を定義する。back prop （=つまりどう微分して逆伝搬するか）は`autograd`を使用することで自動的に定義されるとのこと。学習できるパラメーターは `parameters()`で見ることができる。

In [25]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv2's .weight

10
torch.Size([6, 1, 5, 5])
torch.Size([6])


実際に、乱数を入力して順伝播の後に出力される値を確認する。このチュートリアルで想定している画像の入力サイズは 32×32とのこと（他の、例えば MNIST を使ってみたいならリサイズが必要）。

Conv2dは `in_channels, out_channels, kernel_size,...`を最初３つの引数にとり、ここではそれぞれ、1, 6, 5 と指定されたモデルを考えている。それに合わせるために、$1\times 1\times 32\times 32$ 次元のデータ（sample, channel, x, y) の乱数を作成する必要があることに留意する。

また、PyTorch全ての関数について、ミニバッチ単位での入力を想定している。そのため、single sample （バッチとかの概念がない）はそのまま入力することができない。もし single sample を入力したい場合は、ダミーの batch dimension を入れておく必要があり、`input.squeeze(0)`でできるとのこと。

In [28]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1032, -0.1513,  0.0104,  0.1297, -0.1420,  0.0382,  0.0640,  0.0999,
         -0.0293, -0.0775]], grad_fn=<AddmmBackward>)


ちなみに（もちろん）、サンプル数を変えると出力の形式も変わる（サンプル毎に計算が行われる様子が確認できる）ことも見ておく。

In [30]:
input = torch.randn(2, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0617, -0.1376,  0.0362,  0.1403, -0.1194,  0.0355,  0.0726,  0.0576,
          0.0041, -0.0669],
        [ 0.0824, -0.1423,  0.0140,  0.1287, -0.1344,  0.0554,  0.0512,  0.0616,
         -0.0087, -0.0911]], grad_fn=<AddmmBackward>)


勾配バッファを初期化して（全てのパラメータに対する勾配を0にする）、適当な乱数を逆伝播する（=初期化）。

In [18]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## 損失関数の計算

In [38]:
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output)

target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output (1×10)
print(target)

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([[ 0.0681, -0.1341,  0.0408,  0.1429, -0.1310,  0.0323,  0.0555,  0.0797,
         -0.0390, -0.0598]], grad_fn=<AddmmBackward>)
tensor([[ 1.2042, -1.2118,  0.4264,  0.6443,  0.5190,  1.1912, -1.4897, -1.4398,
          1.1242,  0.9439]])
tensor(1.1675, grad_fn=<MseLossBackward>)


In [39]:
print(type(loss))
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

<class 'torch.Tensor'>


ここまでの流れは以下のようになっている：

```python
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

## 逆伝播

backprop するためには、`backward`を呼ぶだけで良い。呼ぶ前後での値を出力している。

In [21]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0139, -0.0019,  0.0056,  0.0240,  0.0207, -0.0270])


## 重みの更新

以下のように（通常のように）重みは更新される（以下の手法は確率的勾配降下法、SGDと呼ばれるものである）。

$$
w^\prime = w - \eta \times \nabla E
$$

SGDは次のコードで実行される。

In [22]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [48]:
print(net.parameters())
print(type(f))
print(f)

<generator object Module.parameters at 0x1308bf750>
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([ 0.0643, -0.1075,  0.0466,  0.0800, -0.1107,  0.0280,  0.0232, -0.0279,
         0.0584, -0.0922], requires_grad=True)


その他の手法での更新（Nesterov-SGD, Adam, RMSProp, etc.）は以下の様に実行できる。

In [23]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update